In [1]:
import sys
sys.path.append('/Users/ianleefmans/Desktop/stock-sentiment/utils')
from data import Database, ScrapeWSB, Stock
%cd /Users/ianleefmans/Desktop/stock-sentiment

/Users/ianleefmans/Desktop/stock-sentiment


In [2]:
import transformers
import torch
from transformers import BertTokenizer

In [16]:

db = Database()
db.use_database('DB1')

print(db.query('show tables;'))

connection established
[(datetime.datetime(2021, 5, 27, 19, 48, 52),)]
[('COMMENTS',), ('POSTS',), ('STOCKS',)]


In [17]:
print(db.query('''SELECT * FROM STOCKS ;'''))

[('GME', datetime.datetime(2021, 5, 14, 1, 43, 41))]


In [28]:
indexes[0][0]

'nboe4x'

In [30]:
db.query("SELECT TITLE FROM POSTS WHERE POST_ID='nboe4x'")

[("I, too, YOLO'D my retirement account into GME when I switched jobs. PICK ME UP ON THE WAY TO THE MOO",)]

In [35]:
db.query("SELECT TITLE FROM POSTS WHERE POST_ID='{}'".format(indexes[0][0]))[0][0]

"I, too, YOLO'D my retirement account into GME when I switched jobs. PICK ME UP ON THE WAY TO THE MOO"

In [25]:
indexes = db.query('''SELECT POST_ID FROM POSTS;''')

In [19]:
db.query('''SELECT * FROM POSTS ;''')

[('nboe4x',
  'GME',
  "I, too, YOLO'D my retirement account into GME when I switched jobs. PICK ME UP ON THE WAY TO THE MOO",
  1276,
  'wallstreetbets',
  'https://i.redd.it/y6k4lkcjqxy61.jpg',
  122,
  '',
  1620960000.0),
 ('nbjhc1',
  'GME',
  'YOLO: I switched jobs in January and just yolod my retirement into GME',
  1535,
  'wallstreetbets',
  'https://i.redd.it/2k7ba30kqwy61.jpg',
  192,
  '',
  1620950000.0),
 ('naruof',
  'GME',
  'Bought more GME cuz I’m retarded.',
  5178,
  'wallstreetbets',
  'https://www.reddit.com/gallery/naruof',
  431,
  '',
  1620860000.0),
 ('nat3xr',
  'GME',
  'Made 700k in GME in Jan. Then did this',
  270,
  'wallstreetbets',
  'https://i.redd.it/ibx9k7mjwpy61.jpg',
  173,
  '',
  1620870000.0),
 ('naha1d',
  'GME',
  '(O.C.) Keep shorting GME 🤡',
  672,
  'wallstreetbets',
  'https://v.redd.it/xqz6skpukmy61',
  118,
  '',
  1620830000.0),
 ('n9yims',
  'GME',
  'Made a bit off apple and put some back into game stop. We at this side of the world

In [18]:
sample_posts = db.query('''SELECT TITLE FROM POSTS ;''')

In [6]:
sample_txt = sample_posts[0][0]

In [7]:
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [8]:
tokens = tokenizer.tokenize(sample_txt)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f' Sentence: {sample_txt}')
print(f'   Tokens: {tokens}')
print(f'Token IDs: {token_ids}')

 Sentence: I, too, YOLO'D my retirement account into GME when I switched jobs. PICK ME UP ON THE WAY TO THE MOO
   Tokens: ['I', ',', 'too', ',', 'Y', '##OL', '##O', "'", 'D', 'my', 'retirement', 'account', 'into', 'GM', '##E', 'when', 'I', 'switched', 'jobs', '.', 'P', '##IC', '##K', 'ME', 'UP', 'ON', 'THE', 'WA', '##Y', 'TO', 'THE', 'M', '##O', '##O']
Token IDs: [146, 117, 1315, 117, 162, 13901, 2346, 112, 141, 1139, 4406, 3300, 1154, 14748, 2036, 1165, 146, 6759, 5448, 119, 153, 9741, 2428, 22157, 19753, 21748, 7462, 22751, 3663, 16972, 7462, 150, 2346, 2346]


In [9]:
sample_txt

"I, too, YOLO'D my retirement account into GME when I switched jobs. PICK ME UP ON THE WAY TO THE MOO"

In [11]:
import torch
encoding = tokenizer.encode_plus(
    sample_txt,
    #max_length=32,
    add_special_tokens=True, # Add '[CLS]' and '[SEP]'
    return_token_type_ids=False,
    pad_to_max_length=True,
    return_attention_mask=True,
    return_tensors='pt',  # Return PyTorch tensors
)

In [12]:
encoding

{'input_ids': tensor([[  101,   146,   117,  1315,   117,   162, 13901,  2346,   112,   141,
          1139,  4406,  3300,  1154, 14748,  2036,  1165,   146,  6759,  5448,
           119,   153,  9741,  2428, 22157, 19753, 21748,  7462, 22751,  3663,
         16972,  7462,   150,  2346,  2346,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [14]:
tokenizer.convert_ids_to_tokens(encoding['input_ids'][0])

['[CLS]',
 'I',
 ',',
 'too',
 ',',
 'Y',
 '##OL',
 '##O',
 "'",
 'D',
 'my',
 'retirement',
 'account',
 'into',
 'GM',
 '##E',
 'when',
 'I',
 'switched',
 'jobs',
 '.',
 'P',
 '##IC',
 '##K',
 'ME',
 'UP',
 'ON',
 'THE',
 'WA',
 '##Y',
 'TO',
 'THE',
 'M',
 '##O',
 '##O',
 '[SEP]']

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, max_len):
        self.max_len = max_len
        self.PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
        self.tokenizer = BertTokenizer.from_pretrained(self.PRE_TRAINED_MODEL_NAME)
        self.db = Database()
        self.db.use_database('DB1')
        self.indexes = self.db.query('''SELECT POST_ID FROM POSTS;''')
        
        
    def __len__(self):
        return len(self.indexes)
        
    def __getitem__(self, index):
        post = db.query("SELECT TITLE FROM POSTS WHERE POST_ID='{}'".format(indexes[index][0]))[0][0]
        encoding = self.tokenizer.encode_plus(
            post,
            max_length=self.max_len,
            add_special_tokens=True, # Add '[CLS]' and '[SEP]'
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',  # Return PyTorch tensors
            )
        
        return {
            'post': post,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            #'targets': torch.tensor(target, dtype=torch.long)
            }
        
        
        
        
    